Install dependencies.

In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from transformers import BertModel, BertTokenizer
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset, Dataset

/home/ryler/Programming/Projects/Custom-Models/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load in the dataset with latin1 encoding.

In [3]:
df = pd.read_csv("/home/ryler/Datasets/Mcdonalds-Review-Text-Classification/McDonald_s_Reviews.csv", encoding="latin1")
# df = pd.read_csv("/home/rynutty/Documents/DataSets/Mcdonalds-Reviews/McDonald_s_Reviews.csv", encoding="latin1")

Review the features.

In [4]:
df.head()

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star


Get a sense of the size of the dataset.

In [5]:
df.shape

(33396, 10)

I saw someone else modify the ratings label into three classes. 2 = positive rating, 1 = neutral rating, 0 = negative rating. I think this makes a lot more sense than predicting exact start ratings because it's never implied in a review whether the rating would be 4 or 5 starts, but it's easy to tell whether the rating is positive, neutral, or negative.

In [6]:
def clean_ratings(ratings):
    ratings = [int(rate[0]) for rate in ratings]
    cleaned_ratings = []

    for rating in ratings:
        if rating >= 4:
            cleaned_ratings.append(2)
        elif rating == 3:
            cleaned_ratings.append(1)
        else:
            cleaned_ratings.append(0)

    return cleaned_ratings

Split our dataset into evaluation and training.

In [15]:
train_cutoff = int(len(df) * 0.8)

train = df.iloc[:train_cutoff, :]
eval = df.iloc[train_cutoff:, :].reset_index(drop=True)

x_train = train["review"]
y_train = clean_ratings(train["rating"])

x_eval = eval["review"]
y_eval = clean_ratings(eval["rating"])

MLP class just to make the code a little bit easier to read. Not very necessary.

In [ ]:
class MLP(nn.Module):

    def __init__(self, in_features, out_features):
        super().__init__()

        self.inference = nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.ReLU()
        )

    def forward(self, x):
        return self.inference(x)

This is where I made some transformer encoder blocks for contextualising our embeddings, this is what actually processes the data.

The embeddings start off carrying no contextual information, each token only knows about itself. Transformers use a mechanism known as Attention. Attention makes each word in the sentence look at eachother, for example the last word in a sentence may capture the meaning of the entire sentence because it now carries information about all of the words that came before it.

Think of how you would read a paragraph, you start at the beginning knowing nothing, but by the end you've built up enough context so that by the time you reach the last word, you understand the meaning behind the paragraph, that's similar to how transformers behave for NLP tasks.

And once you have that embedding that carries the whole meaning of a sentence or paragraph, you can filter that through a MLP to get a good classification.

In [16]:
class Encoder(nn.Module):

    def __init__(self, d_model, num_heads, num_encoder_blocks):
        super().__init__()

        blocks = [EncoderBlock(d_model=d_model, num_heads=num_heads) for _ in range(num_encoder_blocks)]
        self.inference = nn.Sequential(*blocks)

    def forward(self, x):
        return self.inference(x)
    

class EncoderBlock(nn.Module):

    def __init__(self, d_model, num_heads):
        super().__init__()

        self.layer_norm = nn.LayerNorm(d_model)
        self.mha = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, dropout=0.2, batch_first=True, device="cuda")
        self.mlp = MLP(in_features=d_model, out_features=d_model)

    def forward(self, x):
        x = self.layer_norm(x)
        identity = x
        x, attention_weights = self.mha(x, x, x)
        x = x + identity
        x = self.layer_norm(x)
        identity = x
        x = self.mlp(x)
        x = x + identity

        return x

This model just ties together my encoder with a classification head that takes the contextualised embeddings and makes classification with them.

In [ ]:
class SequenceClassifier(nn.Module):

    def __init__(self):
        super().__init__()
            
        self.encoder = Encoder(d_model=768, num_heads=12, num_encoder_blocks=2)
        self.head = nn.Linear(in_features=768, out_features=3)

    def forward(self, x):
        x = self.encoder(x)
        x = x[:, -1, :]
        x = self.head(x)
        
        return x

I decided to use a pretrained embedding space from bert because I didn't want to take a long time training my own, I didn't let bert process the embeddings, I just got positional encodings and base embeddings.

In [9]:
class Embedder():

    def __init__(self):

        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
        self.model = BertModel.from_pretrained("bert-base-uncased")

        for param in self.model.parameters():
            param.requires_grad = False
    
    def __call__(self, sentences: str):

        if isinstance(sentences, torch.Tensor):
            sentences = list(sentences)
            
        input_ids = self.tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")["input_ids"]
        embeddings = self.model.embeddings(input_ids)

        return embeddings

Dataset class for my datasets.

In [10]:
class ReviewDataset(Dataset):

    def __init__(self, reviews, ratings):
        super().__init__()

        self.reviews = reviews
        self.ratings = ratings
        self.embedder = Embedder()

    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, idx):
        return self.reviews[idx], self.ratings[idx]
    

My model initializaton, optimizer initialization, and loss function initialization. You can also see I moved my model to my GPU for faster training.

In [11]:
model = SequenceClassifier().to("cuda")
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001, weight_decay=0.001)
loss_fn = nn.CrossEntropyLoss()

Created my data loaders with my dataset class so that training becomes more simple, and it batches for me automatically, it also shuffles too which is nice.

In [ ]:
train_dataset = ReviewDataset(reviews=x_train, ratings=y_train)
test_dataset = ReviewDataset(reviews=x_eval, ratings=y_eval)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=True)

Clear anything off of my GPU that we don't need so it doesn't mess with training.

In [13]:
torch.cuda.empty_cache()

Run the training loop for 20 epochs.

In [14]:
epochs = 20
embedder = Embedder()

for epoch in range(1, epochs+1):
    print("Starting Training...")

    running_train_loss = 0
    train_total = 0
    train_correct = 0

    model.train()
    for reviews, ratings in tqdm(train_dataloader):
        embeddings = embedder(reviews).to("cuda")
        ratings = ratings.to("cuda")

        optimizer.zero_grad()
        logits = model(embeddings)

        loss = loss_fn(logits, ratings)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()
        train_total += len(reviews)
        prediction = torch.argmax(logits, dim=1)
        train_correct += (prediction == ratings).sum().item()

    print(f"Avg loss: {running_train_loss / train_total}, accuracy: {train_correct / train_total}")

    running_eval_loss = 0
    eval_total = 0
    eval_correct = 0

    model.eval()
    with torch.no_grad():
        for reviews, ratings in tqdm(test_dataloader):
            embeddings = embedder(reviews).to("cuda")
            ratings = ratings.to("cuda")

            logits = model(embeddings)
            loss = loss_fn(logits, ratings)

            running_eval_loss += loss.item()
            eval_total += len(reviews)
            predicted = torch.argmax(logits, dim=1)
            eval_correct += (predicted == ratings).sum().item()

    print(f"Avg loss: {running_eval_loss / eval_total}, accuracy: {eval_correct / eval_total}")



Starting Training...


100%|██████████| 835/835 [01:25<00:00,  9.79it/s]


Avg loss: 0.01979332622227099, accuracy: 0.7558766282377601


100%|██████████| 209/209 [00:10<00:00, 20.37it/s]


Avg loss: 0.01728005724663506, accuracy: 0.7901197604790419
Starting Training...


...

Avg loss: 0.013015102334097474, accuracy: 0.8422155688622754
Starting Training...


100%|██████████| 835/835 [01:25<00:00,  9.79it/s]


Avg loss: 0.012554030250219693, accuracy: 0.8393472076658183


100%|██████████| 209/209 [00:10<00:00, 20.27it/s]

Avg loss: 0.013161573931574822, accuracy: 0.8423652694610778


Thank you for reading through this if you did, this is my first post on kaggle so hopefully it went alright! I also noticed my model didn't end up overfitting which I love to see.